In [26]:
import requests

END_POINT = 'https://disclosure.edinet-fsa.go.jp/api/v1'
submission_info_endpoint = f'{END_POINT}/documents.json'
submission_request_parameters = {
    'date': '2023-09-08',
    'type': 2
}
submission_info_response = requests.get(
    submission_info_endpoint, submission_request_parameters
)
submission_info_json = submission_info_response.json()


In [27]:
import pandas as pd

raw_submission_info_df = pd.DataFrame(submission_info_json['results'])
raw_submission_info_df.columns

Index(['seqNumber', 'docID', 'edinetCode', 'secCode', 'JCN', 'filerName',
       'fundCode', 'ordinanceCode', 'formCode', 'docTypeCode', 'periodStart',
       'periodEnd', 'submitDateTime', 'docDescription', 'issuerEdinetCode',
       'subjectEdinetCode', 'subsidiaryEdinetCode', 'currentReportReason',
       'parentDocID', 'opeDateTime', 'withdrawalStatus', 'docInfoEditStatus',
       'disclosureStatus', 'xbrlFlag', 'pdfFlag', 'attachDocFlag',
       'englishDocFlag', 'csvFlag', 'legalStatus'],
      dtype='object')

In [28]:
# 重要なカラムに絞る
submission_info_df = raw_submission_info_df[['docID', 'edinetCode', 'secCode', 'filerName', 'docDescription']]
submission_info_df.head()

,docID,edinetCode,secCode,filerName,docDescription
0,S100RTJS,E01045,49660,上村工業株式会社,自己株券買付状況報告書（法２４条の６第１項に基づくもの）
1,S100RK4A,E12444,None,三井住友トラスト・アセットマネジメント株式会社,有価証券報告書（内国投資信託受益証券）－第17期(2022/06/10－2023/06/09)
2,S100RPUU,E09666,None,アムンディ・ジャパン株式会社,有価証券報告書（内国投資信託受益証券）－第7期(2022/06/16－2023/06/15)
3,S100RRHB,E12453,None,ニッセイアセットマネジメント株式会社,有価証券届出書（内国投資信託受益証券）
4,S100RQ2B,E12453,None,ニッセイアセットマネジメント株式会社,半期報告書（内国投資信託受益証券）－第1期(2022/12/09－2023/12/11)


In [29]:
# 有価証券報告書の情報を抽出する。
securities_report_infos = []
for i, row in submission_info_df.iterrows():
    doc_desc = row['docDescription']
    
    if doc_desc is None:
        continue
    
    if ('有価証券報告書' in doc_desc) and ('受益証券' not in doc_desc):
        row_to_dataframe = pd.DataFrame([row])
        securities_report_infos.append(row_to_dataframe)

if len(securities_report_infos) == 0:
    print('有価証券報告書の提出情報がありません。')
else:
    print(f'{len(securities_report_infos)} 件の有価証券報告書が抽出されました。')
    securities_report_info_df = pd.concat(securities_report_infos)

5 件の有価証券報告書が抽出されました。


In [30]:
# 今回はマネーフォーワードの第9期決算情報を対象とする。
docID = 'S100N8ST'
document_endpoint = f'{END_POINT}/documents/{docID}'
document_request_parameters = {
    'type': 1
}
document_response = requests.get(document_endpoint, document_request_parameters)

In [31]:
# まず、返ってきたデータを zip 形式で保存する。
zip_file_full_path = f'./content/{docID}.zip'
with open(zip_file_full_path, 'wb') as f:
    for chunk in document_response.iter_content(chunk_size=1024):
        f.write(chunk)

In [32]:
# zip ファイルを解凍する
import zipfile
import os
os.makedirs(f'./content/{docID}', exist_ok=True)
with zipfile.ZipFile(zip_file_full_path) as zip_f:
    zip_f.extractall(f'./content/{docID}')

In [39]:
# xbrl ファイルを発見する
from glob import glob
# PublicDoc 内に格納されている xbrl ファイルが分析対象となるファイルである。
xbrl_expression = f'./content/{docID}/**/PublicDoc/**/*.xbrl'
xbrl_paths = glob(xbrl_expression, recursive=True)
xbrl_paths

['./content/S100N8ST/XBRL/PublicDoc/jpcrp030000-asr-001_E33390-000_2020-11-30_02_2022-01-18.xbrl']

In [40]:
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser
parser = EdinetXbrlParser()
# Step2で特定した XBRL ファイルのパスを選択
xbrl_path = xbrl_paths[0]
parsed_xbrl = parser.parse_file(xbrl_path)

/opt/homebrew/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [41]:
# この決算情報がいつのものか確かめる。
key = "jpdei_cor:CurrentFiscalYearEndDateDEI"
context_ref = "FilingDateInstant"
report_end_date_info = parsed_xbrl.get_data_by_context_ref(key, context_ref)
report_end_date = report_end_date_info.get_value()
report_end_date

'2020-11-30'

In [42]:
# 「事業の内容」という事業の概要について記述された情報を取得する。
key = "jpcrp_cor:DescriptionOfBusinessTextBlock"
context_ref = "FilingDateInstant"
summary_of_business_info = parsed_xbrl.get_data_by_context_ref(key, context_ref)
summary_of_business = summary_of_business_info.get_value()

In [43]:
# 結果の出力
from IPython.display import HTML
HTML(summary_of_business)